In [1]:
import pickle
import nltk
import os
import math
import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import unicodedata

Using TensorFlow backend.
C:\Users\Dell\New folder\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dell\New folder\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Dell\New folder\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Dell\New folder\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
news = pd.read_csv("News_articles_dataset.csv", parse_dates=['Date'])
news.sort_index(inplace=True)


In [3]:
historical_data = pd.read_csv('HistoricalData_1623748350213.csv', parse_dates = ['Date'])
historical_data.sort_values('Date', inplace=True)
historical_data.reset_index()


,index,Date,Close/Last,Volume,Open,High,Low
0,2515,2011-06-15,$11.6696,395841722,$11.7768,$11.7964,$11.6029
1,2514,2011-06-16,$11.6129,507299317,$11.675,$11.7386,$11.3689
2,2513,2011-06-17,$11.4379,614859874,$11.7496,$11.7589,$11.4057
3,2512,2011-06-20,$11.2614,639595904,$11.3341,$11.3464,$11.0893
4,2511,2011-06-21,$11.6179,492451483,$11.31,$11.6357,$11.2571
...,...,...,...,...,...,...,...
2511,4,2021-06-08,$126.74,74403770,$126.6,$128.46,$126.2101
2512,3,2021-06-09,$127.13,56877940,$127.21,$127.75,$126.52
2513,2,2021-06-10,$126.11,71186420,$127.02,$128.19,$125.94
2514,1,2021-06-11,$127.35,53522370,$126.53,$127.44,$126.1


In [4]:
# Merging the datasets
data = pd.merge(news, historical_data, how='inner', on=['Date'])

In [5]:
# Removing $ sign and converting the string numeric values to float values
data["Close/Last"] = data["Close/Last"].str.replace('$', '').astype(float)
data["Open"] = data["Open"].str.replace('$', '').astype(float)
data["High"] = data["High"].str.replace('$', '').astype(float)
data["Low"] = data["Low"].str.replace('$', '').astype(float)

In [6]:
data.set_index("Date", inplace = True)
data.sort_index(ascending=True,inplace=True)

data.to_csv('News and Historical data append.csv')

data['compound'] = ''
data['negative'] = ''
data['neutral'] = ''
data['positive'] = ''

In [7]:
# instantiating the Sentiment Analyzer
sid = SentimentIntensityAnalyzer()

# calculating sentiment scores
data['compound'] = data['Headlines'].apply(lambda x: sid.polarity_scores(x)['compound'])
data['negative'] = data['Headlines'].apply(lambda x: sid.polarity_scores(x)['neg'])
data['neutral'] = data['Headlines'].apply(lambda x: sid.polarity_scores(x)['neu'])
data['positive'] = data['Headlines'].apply(lambda x: sid.polarity_scores(x)['pos'])

data = data[['Close/Last', 'compound', 'negative', 'neutral', 'positive', 'Open', 'High', 'Low', 'Volume']]
data.to_csv('stock_data.csv')

In [8]:
stock_data = pd.read_csv('stock_data.csv', index_col = False)
# renaming the column
stock_data.rename(columns={'Unnamed: 0':'Date'}, inplace = True)
# setting the column 'Date' as the index column
stock_data.set_index('Date', inplace=True)

In [9]:
# capturing data to be used for each column
close_price = stock_data.iloc[:,0] #close
compound = stock_data.iloc[:,1] #compound

In [10]:
# concatenating the captured training data into a dataframe
need_data = pd.concat([close_price, compound], axis=1)

# setting column names of the revised stock data
need_data.columns = ['close_price', 'compound']  

y = need_data['close_price']
# setting the features dataset for prediction  
cols = ['close_price', 'compound']
X = need_data[cols]

close_data = need_data.filter(['close_price'])

# Convert the dataframe to a numpy array
dataset = need_data.values
close_data = close_data.values

In [11]:
# Get the number of rows to train the model on
# training_data_len = math.ceil(len(dataset)*.8)

scaler1 = MinMaxScaler(feature_range=(0,1))
scaler2 = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler1.fit_transform(dataset)
scaled_close_data = scaler2.fit_transform(close_data)


In [12]:
# Create the training data set
# train_data = scaled_data[0:training_data_len , :]

# Split the data into x_train and y_train data sets
x_train = []
y_train = []
past_days = 20
for i in range(past_days, len(scaled_data)):
    x_train.append(scaled_data[i-past_days:i, :])
    y_train.append(scaled_data[i, 0])


In [13]:
# Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 2))


In [14]:
# Build the LSTM model
lstm = Sequential()
lstm.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 2)))
lstm.add(LSTM(50, return_sequences= False))
lstm.add(Dense(25))
lstm.add(Dense(1))

In [15]:
# Compile the model
lstm.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
lstm.fit(x_train, y_train, batch_size = 16, epochs = 100, verbose=1)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1576/1576 [==============================] - 16s 10ms/step - loss: 0.0071
Epoch 2/100
1576/1576 [==============================] - 4s 2ms/step - loss: 6.5524e-04
Epoch 3/100
1576/1576 [==============================] - 4s 2ms/step - loss: 5.1531e-04
Epoch 4/100
1576/1576 [==============================] - 3s 2ms/step - loss: 5.0787e-04
Epoch 5/100
1576/1576 [==============================] - 3s 2ms/step - loss: 4.8877e-04A: 1
Epoch 6/100
1576/1576 [==============================] - 3s 2ms/step - loss: 4.4935e-04
Epoch 7/100
1576/1576 [==============================] - 3s 2ms/step - loss: 4.2762e-04
Epoch 8/100
1576/1576 [==============================] - 3s 2ms/step - loss: 4.7379e-04
Epoch 9/100
1576/1576 [==============================] - 3s 2ms/step - loss: 5.0655e-04
Epoch 10/100
1576/1576 [==============================] - 4s 2ms/step - loss: 4.4395e-04
Epoch 11/100
1576/15

In [19]:
# # Create the testing data set
# test_data = scaled_data[training_data_len - past_days: , :]

# # Create the data sets x_test and y_test
# x_test = []
# y_test = dataset[training_data_len:, 0]
# for i in range(past_days, len(test_data)):
#     x_test.append(test_data[i-past_days:i, :])

# # Convert the data to a numpy array
# x_test = np.array(x_test)

# # Reshape the data
# x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 2))

# # Get the models predicted price values
# predictions = model.predict(x_test)
# predictions = scaler2.inverse_transform(predictions)

# Getting next day from where the Historical Dataset ends
# last_day = data.index[-1]
# next_day = last_day + datetime.timedelta(days = 1)

# # Create a new dataframe
# new_df = data.filter(['Close/Last', 'compound'])

# # Get the last 20 day closing price values and convert the dataframe to an array
# last_20_days = new_df[-20: ].values

# # Scale the data to be values between 0 and 1
# last_20_days_scaled = scaler1.transform(last_20_days)

# # Create an empty list
# X_test = []

# # Append the past 20 days
# X_test.append(last_20_days_scaled)

# # Convert the X_test data set to a numpy array
# X_test = np.array(X_test)

# # Reshape the data
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))

# # Get the predicted scaled price
# pred_price = model.predict(X_test)

# # undo the scaling
# pred_price = scaler2.inverse_transform(pred_price)
# print('Predicted Close Price for {} '.format(next_day) + ' = ', pred_price)


pickle.dump(lstm, open('model.pkl', 'wb'))

model = pickle.load(open('model.pkl', 'rb'))
#print(model.predict([[[132.54, 0.9926], [127.85, 0.9900], [128.10, 0.9915], [129.74, 0.9611], [124.97, 0.9958], [127.45, 0.9877], [126.27, 0.9972], [124.85, 0.9989], [124.69, 0.9983], [127.31, 0.9974], [125.43, 0.9798], [127.10, 0.9951], [126.90, 0.9973], [126.85, 0.9965], [125.28, 0.9986], [124.61, 0.9937], [124.28, 0.9976], [125.06, 0.7719], [123.54, -0.3348], [125.89, 0.9801]]]))